# Running an ONNX model
## Note that this method does not currently work.
In this section I try out loading and running an ONNX model.

First we load and czech the model.

In [5]:
import onnx
path = "../onnx/default/potsawee.onnx"
onnx_model = onnx.load(path)
onnx.checker.check_model(path)

create inference session with `ort.infernnce`

An inference session is, idk.

In [6]:
import torch
import onnxruntime as ort
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

sample_text = "In the beginning, God created the heavens and the earth."
inputs = tokenizer(sample_text, return_tensors="pt")
decoder_input_ids = torch.ones_like(inputs["input_ids"])

path = "../onnx/default/potsawee.onnx"
output_names = ["output"]
dynamic_axes = {"input_ids": {0: "batch_size", 1: "seq_len"}, "attention_mask": {0: "batch_size", 1: "seq_len"}, "output": {0: "batch_size", 1: "seq_len"}}


session = ort.InferenceSession(path)


In [7]:
print(session.get_inputs()[0].name)
print(session.get_inputs()[1].name)
print(session.get_outputs()[0].name)

input_ids
attention_mask
output


In [8]:
outputs = session.run(output_names, {
  'input_ids': inputs["input_ids"].numpy(), 
  'attention_mask':  inputs["attention_mask"].numpy()
  })


ValueError: Model requires 3 inputs. Input Feed contains 2

In [ ]:
print("outputs: ", outputs)


In [ ]:
result = outputs[0].argmax(axis=1)+1
print("result: ", result)
question_answer = tokenizer.decode(result[0], skip_special_tokens=False)
print(question_answer)
